# Molecular Dynamics Simulations : Part 2

We have been slowly building up our knowledge of molecular simulations. We first simulated a Lennard Jones fluid using Monte Carlo, then we switched our simulation method to a molecular dynamics simulation.

This week, we will use a research-grade molecular dynamics software package called OpenMM.
Because we are still running simulations on our laptops, we will still be limited to small systems.
This week, we will simulate systems of alkanes in water boxes.

We will analyze the trajectory by viewing the molecule using NGLView. 
We'll also analyze behavior using MDAnalysis.

<div class="alert alert-block alert-warning"> 
<h2>Molecular Dynamics System Size</h2>
    
Even though these simulations are much larger and the system more complicated than the one we ran last week, these are still much, much, smaller than typical MD simulations run these days. This is because we need to run these on our laptops in a reasonable amount of time.

In this simulation, we have 1186 atoms. **Question** - How does this compare to our simulation size from last week?
</div>

## System Preparation

Dr. Nash set up this simulation systems using many pieces of software including `tleap`, part of AmberTools Software Suite, and PackMol, a command-line software for buliding mixtures of molecular systems. 
If you are curious about the steps for creating the systems, see the scripts `build_sys.sh` in the `octane` and `carbitol` folders

We are starting our simulation this time from files with the extensions `prmtop` and `inpcrd`. These contain forcefield parameters (you get to read about this in the next section) and initial coordinates for our simulation system.

The next cell shows our starting simulation structure using NGLView and MDAnalysis.

In [ ]:
import nglview as nv
import MDAnalysis as mda

molecule = "carbitol"
resname = "CAR"

u = mda.Universe(f"water_{molecule}.prmtop", f"water_{molecule}.inpcrd")

view = nv.show_mdanalysis(u)
view.clear_representations()

# If curious about molecular represenations ("line", "hyperball"), see here: 
# https://nglviewer.org/ngl/api/manual/molecular-representations.html
view.add_representation("line", "WAT", opacity=0.50)
view.add_representation("hyperball", "not WAT")
view

## Running a Simulation

The cell below gives an OpenMM script for running a molecular dynamics simulation on our structure.
We choose a timestep representing 2 femtoseconds.

**While you are waiting for your simulation to run, read the information in the next cell about forcefields.
If the simulation is still running when you are done, move on to the programming questions at the end of the lab.**

In [ ]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
from sys import stdout


# Load Amber topology and coordinate files
prmtop = app.AmberPrmtopFile(f"water_{molecule}.prmtop")
inpcrd = app.AmberInpcrdFile(f"water_{molecule}.inpcrd")

# Create OpenMM system from the prmtop file
# Apply PME for long-range electrostatics and constraint hydrogen-heavy atom bonds
system = prmtop.createSystem(nonbondedMethod=app.PME, 
                             nonbondedCutoff=0.9*unit.nanometers, 
                             constraints=app.HBonds)

# Add a barostat to maintain pressure at 1 atm
pressure = 1 * unit.atmospheres
temperature = 300 * unit.kelvin
barostat = mm.MonteCarloBarostat(pressure, temperature)
system.addForce(barostat)

# Set up the integrator (Langevin with friction for temperature control)
friction_coefficient = 1 / unit.picosecond  # Friction coefficient (gamma)
timestep = 2.0 * unit.femtoseconds  # Time step for integration
integrator = mm.LangevinIntegrator(temperature, friction_coefficient, timestep)

# Create the simulation object
simulation = app.Simulation(prmtop.topology, system, integrator)

# Set initial positions from Amber inpcrd file (will set box vectors if present)
simulation.context.setPositions(inpcrd.positions)

# Set periodic box vectors if they exist in the inpcrd file
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

# Run an NPT simulation for 200 ps
n_steps = 250000  # 300 ps / 2 fs per step = 150,000 steps

# Minimize the energy
print("Minimizing energy...")
simulation.minimizeEnergy()

# Save the minimized structure to a PDB file for inspection
minimized_positions = simulation.context.getState(getPositions=True).getPositions()
with open('minimized_system.pdb', 'w') as f:
    app.PDBFile.writeFile(prmtop.topology, minimized_positions, f)

# Set reporters to log simulation progress and trajectory data
simulation.reporters.append(app.StateDataReporter(stdout, 
                                                  1000, 
                                                  step=True, 
                                                  potentialEnergy=True, 
                                                  temperature=True, 
                                                  density=True))  # Report every 1000 steps

# Add a StateDataReporter to save state data to a file
simulation.reporters.append(app.StateDataReporter(f"state_data_{molecule}.txt", 
                                                  1000,  # Save every 1000 steps
                                                  step=True, 
                                                  potentialEnergy=True, 
                                                  kineticEnergy=True,
                                                  totalEnergy=True,
                                                  temperature=True,
                                                  volume=True,
                                                  density=True,
                                                  progress=True,
                                                  remainingTime=True,
                                                  speed=True,
                                                  totalSteps=n_steps,
                                                  separator=','))  

simulation.reporters.append(app.DCDReporter(f'trajectory_{molecule}.dcd', 500))  # Save trajectory every 500 steps (every picosecond)


print("Running NPT simulation...")
simulation.step(n_steps)

# Save the final positions
final_positions = simulation.context.getState(getPositions=True).getPositions()
with open("final_system.pdb", "w") as f:
    app.PDBFile.writeFile(prmtop.topology, final_positions, f)

print("Simulation complete.")


## Modeling More Complicated Systems - Extending the "force field"

<div class="alert alert-block alert-info"> 
<h3>Do I need to understand this?</h3>
    
The information in this box is provided to give you a deeper understanding of how molecular dynamics simulations work.
However, you do not need to understand all of it, as Dr. Nash has set up the system with appropriate forcefield parameters for you!
</div>

You may have noticed that our simulation this time is much more complicated than our previous simulations. 
For example, we now have multiple atom types and we have bonds in our system.

This means that our expression for the potential energy has more terms.  
In molecular mechanics, the full expression of this model is often called a **force field**  and is a function which describes the potential energy of a system. 
In our previous simulations, we could describe our system energy using the Lennard Jones equation only. 
With a more complicated system, this is no longer the case.
This force field describes the energy associated with molecular movements such as bond stretching, angle bending, or dihedral angle rotation.  
The force field describes the energies associated with these movements mathematically. The potential energy is commonly represented by the letter $U$, and the total potential energy or "force field" is the sum of terms related to bond stretching, angle bending, torsional rotation (that cis-trans isomerism), electrostatic interaction, and nonbonded Van der Waals interactions.

$$ U = U_{bond} + U_{angle} + U_{torsion} + U_{elec} + U_{vdw} $$

Below we show commonly used forms from these terms.

### Van der Waals Interactions:

This is the term we are most familiar with. It is the only type of term we have to use for a Lennard Jones fluid.

$$ U_{vdw} = 4\epsilon_{ij}[(\frac{\sigma_{ij}}{r_{ij}})^{12} - (\frac{\sigma_{ij}}{r_{ij}})^{6}] $$

These are non-covalent interactions that occur between all atoms and molecules, irrespective of their charge.
$U_{vdw}$ quantifies these weak forces that primarily consist of attractions and repulsions due to transient fluctuations in electron density around atoms.
The equation is formulated using the Lennard-Jones potential, which models the balance between attraction at longer distances (represented by the $(\frac{\alpha_{ij}}{r_{ij}})^{6}$ term) and repulsion at shorter distances (captured by the $(\frac{\alpha_{ij}}{r_{ij}})^{12}$ term).
Here, $\epsilon_{ij}$ provides the depth of the potential well, indicating the strength of the attraction, and $\alpha_{ij}$ represents the finite distance at which the inter-particle potential is zero.
As two atoms approach each other, they first attract weakly, then strongly, but once they get too close, they experience a strong repulsive force.

### Bonds and Angles:

$$ U_{bond} = \frac{1}{2}k_{l}(l-l_{eq})^2 $$

$$ U_{angle} = \frac{1}{2}k_{\theta}(\theta - \theta_{eq})^2 $$

Note that both bonds and angles are commonly described using a harmonic potential (also used to describe spring-mass systems.) 
This should look familiar to your last homework assignment!
The variable $l$ represents the distance between two bonded atoms. The parameters $k_{l}$ and $l_{eq}$ represent the bond stiffness and equilibrium bond length respectively. 
For example, two double bonded carbons will have a shorter bond length ($l_{eq}$) and stiffer bond (stiffer spring, or higher $k_{l}$) than single bonded carbons.

### Torsional (Dihedral) Interactions:

$$ U_{torsion} = \sum_{n=1}^{n_{max}}{U_{n}[1+cos(n\phi-\gamma_{n})]} $$

The torsion energy, represented by $U_{torsion}$, describes the energy changes associated with the rotation around a particular bond, often visualized between four bonded atoms in sequence.
This term is especially significant in cases like rotatable single bonds where molecules can adopt different conformations based on the dihedral angles.
The variable $\phi$ represents the torsional (or dihedral) angle, which is the angle between two intersecting planes formed by three atoms each.
The parameters $U_{n}$ and $\gamma_{n}$ provide amplitude and phase shift respectively.
This trigonometric expression captures the periodic nature of torsional potentials, indicating that after completing a full rotation (360°), the molecule returns to its original state.

### Electrostatic Interactions:

$$ U_{elec} = k_{e}\frac{q_{i}q_{j}}{r_{ij}} $$

The term $U_{elec}$ encapsulates the interactions between charged species within a molecular system.
As the name suggests, it's based on the fundamental principles of electrostatics: opposite charges attract, while like charges repel.
The variables $q_{i}$ and $q_{j}$ represent the charges on atoms i and j, respectively, and $r_{ij}$ is the distance between them.
The constant $k_{e}$ is a proportionality factor that relates to the physical constants of the system.
In essence, this term is a manifestation of Coulomb's Law at the molecular level, accounting for the forces exerted between charged entities in the molecule.




## Viewing Simulation Results

After we've run our simulation, we can peform analysis.
First, we'll watch an animation of our simulation using NGLView.
This week, we add some more complex syntax for selecting parts of our simulation.
In the movie below, we add water molecules using the "line" representation and make them partially transparent.
Our peptide is shown with the "hyperball" representation.

You might try [viewing the documentation](https://nglviewer.org/ngl/api/manual/molecular-representations.html) to pick other representation styles.

In [ ]:
import nglview as nv
import MDAnalysis as mda

u = mda.Universe(f"water_{molecule}.prmtop", f"trajectory_{molecule}.dcd")

In [ ]:
view = nv.show_mdanalysis(u)
view.clear_representations()

# If curious about molecular represenations ("line", "hyperball"), see here: 
# https://nglviewer.org/ngl/api/manual/molecular-representations.html
view.add_representation("hyperball", "WAT", opacity=1.0)
view.add_representation("spacefill", "not WAT")
view.add_representation("unitcell")
view

## Visualizing the system with periodic boundaries

In [ ]:
from util import build_supercell

replicated_universe = build_supercell(u)

# Visualize the new replicated universe with NGLView
view = nv.show_mdanalysis(replicated_universe)
view.clear_representations()

# Add your representations
view.add_representation("hyperball", selection="WAT", opacity=0.50, color="lightblue")
view.add_representation("spacefill", selection="not WAT")

# Show the view
view


## Solvation Analysis

We can use the radial distribution function for a more quantitative analysis of the behavior.
The cell below calculates the radial distribution functions for carbon atoms with other carbon atoms that are not in the same molecule
and for carbon atoms with oxygen atoms in water.


In [ ]:
import mdtraj as md
import matplotlib.pyplot as plt

# Load the trajectory and topology into MDTraj
md_traj = md.load(f"trajectory_{molecule}.dcd", top=f"water_{molecule}.prmtop")

# select only the carbon atoms in octane and oxygen atoms in water
carbon_atoms = md_traj.topology.select(f'resname {resname} and element C')
oxygen_atoms = md_traj.topology.select('resname HOH and element O')  # Adjust if water is named differently

# define atom pairs for Carbon-Carbon RDF, excluding atoms from the same molecule
carbon_pairs = []
for i, atom_i in enumerate(carbon_atoms):
    res_i = md_traj.topology.atom(atom_i).residue
    for j, atom_j in enumerate(carbon_atoms):
        res_j = md_traj.topology.atom(atom_j).residue
        if res_i != res_j:  # Exclude pairs from the same molecule (residue)
            carbon_pairs.append((atom_i, atom_j))

# define atom pairs for Carbon-Oxygen RDF (Carbon in octane, Oxygen in water)
carbon_oxygen_pairs = []
for atom_c in carbon_atoms:
    for atom_o in oxygen_atoms:
        carbon_oxygen_pairs.append((atom_c, atom_o))

# compute the RDF for carbon-carbon and carbon-oxygen pairs
rdf_r_cc, rdf_value_cc = md.compute_rdf(md_traj, pairs=carbon_pairs, r_range=(0.0, 1.0), bin_width=0.01, periodic=True)
rdf_r_co, rdf_value_co = md.compute_rdf(md_traj, pairs=carbon_oxygen_pairs, r_range=(0.0, 1.0), bin_width=0.01, periodic=True)

# plotting
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

ax.plot(rdf_r_cc, rdf_value_cc, label="Carbon-Carbon RDF", color="#2565E8")

ax.plot(rdf_r_co, rdf_value_co, label="Carbon-Oxygen RDF", color="#E82626")

ax.set_ylabel(r'$g(r)$', fontsize=16)
ax.set_xlabel(r'Distance (nm)', fontsize=16)
ax.tick_params(axis="both", which="major", labelsize=12)
ax.legend(fontsize=12)

plt.savefig(f"{molecule}_rdf.png", dpi=300)

## Questions

### Programming Questions

OpenMM heavily uses an object-oriented paradigm. We will see this tomorrow in our lab.
View the [OpenMM repository](https://github.com/openmm/openmm) and answer the following questions.
1. What language or languages is OpenMM written in? Why do you think this is? Did you notice any performance differences compared to the Lennard Jones calculator we used in ASE last week?
2. Read the C++ [harmonic bond class](https://github.com/openmm/openmm/blob/a97bbeb40f60acefebb8a6feb0e2522c758d0db0/openmmapi/include/openmm/HarmonicBondForce.h#L50) in OpenMM. Using what you have learned about C++ classes already, explain the attributes and methods present in the class. Identify the construtor. Identify public and private variables and methods. What does this class represent? Is there anything in the class that you don't understand? 
3. Read the Python [topology class](https://github.com/openmm/openmm/blob/e2453f5ec1fc1fd7916e0b398033c3e9ed341877/wrappers/python/openmm/app/topology.py#L70) and perform the same exercise. Using what you have learned about Python classes, identify different aspects of the class. Where is the constructor defined and wht does it do? What are the class attributes and methods? Is there anything in the class that you don't understand?
### Molecular Dynamics Exercises